In [ ]:
# Exercise 1
# (c)

In [ ]:
# Exercise 2
# (d)

In [ ]:
# Exercise 3
# (c)

In [ ]:
# Exercise 4
# Regularization is a regression formula that pulls the esimates of the coefficiant towards zero. 
# It is useful for a machine learning model that lowers the chance of "over-fitting".

In [1]:
# Exercise 5
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.preprocessing import MinMaxScaler

s3 = boto3.resource('s3')
bucket_name = 'data-445-timlincoln'
bucket = s3.Bucket(bucket_name)

file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# Reading the data-file
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [2]:
# Dropping N/A variables
heart = heart.dropna()

In [3]:
# Defining the input and target
X = heart.drop(columns = ['TenYearCHD'], axis = 1)
Y = heart['TenYearCHD']

# Defining the folds
kf = KFold(n_splits = 5, shuffle = True)

# Defining the lists to store results
mse1 = list()
mse2 = list()

for train_idx, val_idx in kf.split(X):
    
    # Splitting the data
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    Y_train, Y_val = Y.iloc[train_idx], Y.iloc[val_idx]
    
    ############
    # Model 1 ##
    ############
    
    X1 = X_train[['age', 'currentSmoker', 'totChol', 'sysBP', 'diaBP', 'BMI', 'heartRate', 'glucose']]
    X_val_1 = X_val[['age', 'currentSmoker', 'totChol', 'sysBP', 'diaBP', 'BMI', 'heartRate', 'glucose']]
    
    # Transforming the data
    scaler = MinMaxScaler()
    X1 = scaler.fit_transform(X1)
    X_val_1 = scaler.fit_transform(X_val_1)
    
    md1 = LogisticRegression().fit(X1, Y_train)
    
    # Predicting on the validation set
    pred1 = md1.predict(X_val_1)
    
    # Changing predictions to labels
    md1_labels = np.where(pred1 < 0.25, 0, 1)
    
    # Storing the mse
    mse1.append(f1_score(Y_val, md1_labels))
    
    ############
    # Model 2 ##
    ############
    
    X2 = X_train[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate', 'glucose']]
    X_val_2 = X_val[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate', 'glucose']]
    
    # Transforming the data
    X2 = scaler.fit_transform(X2)
    X_val_2 = scaler.fit_transform(X_val_2)
    
    md2 = LogisticRegression().fit(X2, Y_train)
    
    # Predicting on the validation set
    pred2 = md2.predict(X_val_2)
    
    # Changing predictions to labels
    md2_labels = np.where(pred2 < 0.25, 0, 1)
    
    # Storing the mse
    mse2.append(f1_score(Y_val, md2_labels))
    

In [6]:
print('The average F1 score for model 1 is', np.mean(mse1))
print('The average F1 score for model 2 is', np.mean(mse2))

The average F1 score for model 1 is 0.15924596497110682
The average F1 score for model 2 is 0.04030960238881031


In [ ]:
# From the results of both averages of F1 scores, in model 1 the F1 score is larger than the 2nd model
# at 0.159 where model 2 is only 0.040. So with these results, the model to use is model 1.